In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np

from itertools import combinations_with_replacement, combinations
from utils.plotting import aligned_imshow_cbar
from utils.models import IsoelasticWealthChange, Gamble, GamblePair, IsoelasticAgent

mpl.rcParams["font.size"] = 15

In [ ]:
def find_eta_equivalence(g1, g2, x, eta_l=-10, eta_r=10, precision=1e-7):
    eta_range = [eta_l, eta_r]
    while True:
        l = IsoelasticAgent(eta_range[0], x).gamble_difference(g1, g2)
        r = IsoelasticAgent(eta_range[1], x).gamble_difference(g1, g2)
        eta_m = (eta_range[0] + eta_range[1]) / 2
        
        # both signs are the same, i.e., strict preference regardless of eta
        if np.sign(l * r) == 1:
            return np.sign(l) * np.inf

        # we converged at the solution
        if np.abs(eta_range[0] - eta_range[1]) < precision:
            return eta_m      
        
        # improve range
        m = IsoelasticAgent(eta_m, x).gamble_difference(g1, g2)
        if np.sign(m) == np.sign(l):
            eta_range[0] = eta_m
        else:
            eta_range[1] = eta_m

In [ ]:
# Settings
wealth = 10                                # initial wealth
eta_dynamic = 0.5                          # gamble dynamics
eta_bounds = [-1, 1]                       # indifferent eta bounds
gamma_range = np.linspace(-3, 3, 51)       # gamble space sampling
ref_gamble_idx = [25, 25]    # null
# ref_gamble_idx = [13, 37]  # mixed avg
# ref_gamble_idx = [0, 50]   # mixed max
# ref_gamble_idx = [37, 37]  # sure win
# ref_gamble_idx = [13, 13]  # sure loss
# ref_gamble_idx = [25, 37]  # win avg or 0
# ref_gamble_idx = [25, 50]  # win max or 0
# ref_gamble_idx = [13, 25]  # lose avg or 0
# ref_gamble_idx = [0, 25]   # lose max or 0

# Fractals & reference gamble
fractals = [IsoelasticWealthChange(g, eta=eta_dynamic) for g in gamma_range]
n_fractals = len(fractals)
ref_g = Gamble(fractals[ref_gamble_idx[0]], fractals[ref_gamble_idx[1]])

eta_indiff = np.zeros((n_fractals, n_fractals))
for i in range(n_fractals):
    for j in range(n_fractals):
        g = Gamble(fractals[i], fractals[j])
        eta_indiff[i, j] = find_eta_equivalence(
            g1=g, 
            g2=ref_g,
            x=wealth, 
            eta_l=eta_bounds[0], 
            eta_r=eta_bounds[1]
        )
        
fig, (ax1, ax2) = plt.subplots(
    figsize=(10, 10), 
    nrows=2,
    gridspec_kw={'height_ratios': [3, 1]}
)

# Top panel (indifference eta)
im = ax1.imshow(eta_indiff, clim=eta_bounds, cmap="RdBu_r", origin="upper")
ax1.plot(*ref_gamble_idx, 'x', ms=15, mec='y', mew=3)
ax1.plot([-.5, n_fractals -.5], [-.5, n_fractals -.5], "k:", alpha=0.3)
aligned_imshow_cbar(ax1, im)

gamma_mid = (gamma_range[0] + gamma_range[-1]) / 2
gamma_step = gamma_range[1] - gamma_range[0]

# Ticks & labels
ax1.set_xlabel("growth rate $\gamma_1$")
ax1.set_ylabel("growth rate $\gamma_2$")
ticks = np.linspace(0, n_fractals - 1, 3)
tickvalues = (gamma_range[0], gamma_mid, gamma_range[-1])
ticklabels = [f"{g:.2f}" for g in tickvalues]
ax1.set_xticks(ticks)
ax1.set_yticks(ticks)
ax1.set_xticklabels(ticklabels)
ax1.set_yticklabels(ticklabels)
ax1.set_title(f"$\eta^*$ (dynamic={eta_dynamic})")

# Bottom panel
plt.tight_layout()
pos = ax2.get_position()
pos.x0 = pos.x0 + .12
pos.x1 = pos.x1 - .16
ax2.set_position(pos)

ax2.plot(gamma_range, [f(wealth) for f in fractals], ".-", color="b")
ax2.axhline(wealth, color="k")

ax2.set_xlim([gamma_range[0] - gamma_step / 2, gamma_range[-1] + gamma_step / 2])
ax2.set_xticks(tickvalues)
ax2.set_ylabel("wealth [DKK]")
ax2.set_xlabel("growth rate $\gamma$")

plt.show()

In [ ]:
# $\Delta\gamma$ heuristic

In [ ]:
# Experimental setup
fractals = [IsoelasticWealthChange(gamma=g, eta=0) for g in np.linspace(-1, 1, 9)]
gambles = [Gamble(f1, f2) for f1, f2 in combinations_with_replacement(fractals, 2)]
gamble_pairs = [GamblePair(g1, g2) for g1, g2 in combinations(gambles, 2)]

In [ ]:
gamma_dist_thr_perc_range = np.linspace(0, 1, 17)

df = pd.DataFrame(index=gamma_dist_thr_perc_range)

for gamma_dist_thr_perc in gamma_dist_thr_perc_range:
    gamma_dist_thr = gamma_dist_thr_perc * 2
    gamble_pairs_filtered = [gp for gp in gamble_pairs if gp.gamma_distance <= gamma_dist_thr]
    
    # Store values
    df.loc[gamma_dist_thr_perc, "n"] = len(gamble_pairs_filtered)
    df.loc[gamma_dist_thr_perc, "n_nobrainers"] = sum(
        [gp.is_nobrainer for gp in gamble_pairs_filtered])
    df.loc[gamma_dist_thr_perc, "n_mixed"] = sum(
        [gp.is_mixed for gp in gamble_pairs_filtered])
    df.loc[gamma_dist_thr_perc, "n_mixed_nobrainers"] = sum(
        [gp.is_mixed and gp.is_nobrainer for gp in gamble_pairs_filtered])
    df.loc[gamma_dist_thr_perc, "n_win"] = sum(
        [gp.is_win for gp in gamble_pairs_filtered])    
    df.loc[gamma_dist_thr_perc, "n_loss"] = sum(
        [gp.is_loss for gp in gamble_pairs_filtered])

In [ ]:
mpl.rcParams["font.size"] = 15
fig, ax = plt.subplots(figsize=(10, 6))

ax.step(df.index, df["n"], where="post", label="all pairs")
ax.step(df.index, df["n_nobrainers"], where="post", label="nobrainers")
ax.step(df.index, df["n_mixed"], where="post", label="mixed")
ax.step(df.index, df["n_win"], where="post", label="win/loss")

ax.set_ylim([0, 1000])
ax.set_xlim([0, 1])

ax.set_xlabel(r"$\Delta\gamma$ [%]")
ax.set_ylabel(r"# of gamble pairs")

plt.legend()

In [ ]:
mpl.rcParams["font.size"] = 15
fig, ax = plt.subplots(figsize=(10, 6))

ax.step(df.index, df["n_mixed"], where="post", label="mixed", color="k", ls="--")
ax.step(df.index, df["n_mixed_nobrainers"], where="post", label="mixed & nobrainers")
ax.step(df.index, df["n_mixed"] - df["n_mixed_nobrainers"], where="post", label="mixed & unique")

ax.set_ylim([0, 180])
ax.set_xlim([0, 1])

ax.set_xlabel(r"$\Delta\gamma$ [%]")
ax.set_ylabel(r"# of gamble pairs")

plt.legend()